# Predicting Areas of Affluence using Yelp Pricing Data

#### Authors: 
- Eddie Yip [LinkedIn](https://www.linkedin.com/in/eddie-yip-2a37324b/) | [Medium](https://medium.com/@eddie.yip2)
- Hadi Morrow [LinkedIn](https://www.linkedin.com/in/hadi-morrow-4b94164b/) | [GitHub](https://github.com/HadiMorrow) | [Medium](https://medium.com/@hadi.a.morrow)
- Mahdi Shadkam-Farrokhi: [GitHub](https://github.com/Shaddyjr) | [Medium](https://medium.com/@mahdis.pw) | [http://mahdis.pw](http://mahdis.pw)

## Problem Statement [Hadi]

While affluence should never be a factor when choosing to provide disaster aid or not, we must consider the following:

- On the assumption that affluence plays a role, one might relate affluency to preparedness. Those who can afford to will always look out for their families at any cost. Those who can not might not be able to prepare as well due to the fact that it is not an option. 

- On the assumption that affluence is not part of a majority class, if we should be miopic with our search efforts we might want to consider saving the masses, those living in tight coridors and those with little to no income. If effect those most suseptible to losing their lives in a major disaster. 

- Using tax data we aim to show that using YELP data dollar signs is enough to predict where we might want to quickly and accuratly align our efforts. 

New Light Technologies as our audience, we hope to show that while using expensive and hard to handle data such as tax data can be more precise, a quick and dirty aproach could be to simply sord though the dollar signs data on yelp. 

---
[Hadi] - Excellent write up! Here are some suggestions.
1. It would be nice for the reader if we define 'affluence' here at the start. What do we consider "affluent" in our data (I think we mentioned 15% of the area code?)?
2. As a reader, it would be VERY compelling to have an actual case where a natural disaster occured and the affluent areas weren't affected. If possible, research 1 or 2 cases when affluent areas were better prepared for natural disasters - this will help prove our predictive model has a real use case.
3. Tying into the use case, it might be helpful to mention a realistic disaster scenario when only having Yelp! price data would be useful. Like, there's an emergency and there's little time to pull granular information about the area, but knowing the yelp reviews for an area, allows first-responders to know which areas their efforts will have the most impact
4. We need to mention which metric we're going to use and why

## Executive Summary [Mahdi]

- Difficulty gathering data
- Prompt confusing regarding "affluence"
- Other projects used outside data as metric
- We pulled from API and didn't use old data, which was challenging

## Table of Contents
- [Gathering Data](#Gathering-Data)
- [Loading Data](#Loading-Data)
- [Preliminary Exploratory Data Analysis](#Preliminary-Exploratory-Data-Analysis)
- [Cleaning the Data](#Cleaning-the-Data)
- [Feature Engineering](#Feature-Engineering)
- [Exploratory Data Analysis](#Exploratory-Data-Analysis)
- [Model Preparation](#Model-Preparation)
- [Model Selection](#Model-Selection)
- [Model Evaluation](#Model-Evaluation)
- [Conclusions and Recommendations](#Conclusions-and-Recommendations)
- [Source Documentation](#Source-Documentation)

## Gathering Data
We got Yelp data using the API - link 
 
We got IRS data using - source [Eddie]

## Loading Data
- [All]
- For map visual, will need [Basemap](https://rabernat.github.io/research_computing/intro-to-basemap.html)
[source 2](https://jakevdp.github.io/PythonDataScienceHandbook/04.13-geographic-data-with-basemap.html)

In [1]:
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import columnExpander
from ast import literal_eval
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, LogisticRegression
# from mpl_toolkits.basemap import Basemap # this model has messy import process

random_state = 6988

In [ ]:
data_file_path = "./data/total_merge_2.csv"
df_yelp = pd.read_csv(data_file_path, index_col = 0)
df_yelp.reset_index(drop=True, inplace = True) # same indeces were merged using multiple API calls

In [ ]:
df_yelp.shape

## Preliminary Exploratory Data Analysis
- [All]

In [ ]:
df_yelp.head()

In [ ]:
sum_null = df_yelp.isnull().sum()
sum_null[sum_null > 0]

We have many missing values in the data, however many of the columns are not meaningful for our problem and these columns can be safely dropped.

Also, `categories`, `location`, and `transactions` are compressed data columns and will need to be unpacked.

## Cleaning the Data
- [Mahdi] one person for consistency

### Yelp Price

In [ ]:
df_yelp["price"].isnull().sum()

We decided to drop null prices from analysis as this is the key indicator we're looking to predict with.

In [ ]:
df_yelp.dropna(subset=["price"], inplace = True)

In [ ]:
df_yelp.shape

### Converting Yelp Price to ordinal values

In [ ]:
df_yelp['price'] = df_yelp['price'].map({'$': 1, '$$': 2, '$$$': 3, '$$$$':4})

In [ ]:
df_yelp['price'].value_counts()

### Dropping unneccessary columns

In [ ]:
keepers = ['categories','id', 'location', 'price', 'rating', 'review_count', 'transactions', 'coordinates']
df_yelp = df_yelp[keepers]

### Parsing location data

In [ ]:
def get_keys_from_sting_dict(string, keys):
    if len(string) == 0:
        return None
    dic = literal_eval(string)
    out = {}
    for key in keys:
        out[key] = dic.get(key)
    return out

In [ ]:
location = "location"
keys = ["zip_code", "city", 'state']
zips_and_cities = df_yelp[location].map(lambda string: get_keys_from_sting_dict(string, keys))

for key in keys:
    df_yelp[key] = [pair[key] for pair in zips_and_cities]
    
df_yelp.drop(columns=[location], inplace = True)

### Filtering for NYC-only

#### Removed non-NY state

In [ ]:
df_yelp = df_yelp[df_yelp['state'] == "NY"]

#### Imputing missing zip codes

In [ ]:
df_yelp[df_yelp['zip_code'] == ""]

All of these locations were found using Google Maps; their zip codes were are input manually

In [ ]:
df_yelp.loc[df_yelp["id"] == "ECY0sIYxPJio81dteqiMhg","zip_code"] = "10007"
df_yelp.loc[df_yelp["id"] == "6u5cnsN35mJz24HMQ9pfFw","zip_code"] = "10314"
df_yelp.loc[df_yelp["id"] == "BilbRcNQXKmcBFvLm4gxAQ","zip_code"] = "11372"
df_yelp.loc[df_yelp["id"] == "jZzbV6SRt9FXdCoziNv5xw","zip_code"] = "11373"
df_yelp.loc[df_yelp["id"] == "9XXQ2w3DCFytsjjb3KmU7g","zip_code"] = "10017"

### Remove by NYC zip
We used the range of zip codes designated for NYC - [source](https://www.nycbynatives.com/nyc_info/new_york_city_zip_codes.php)

In [ ]:
min_zip = 10001
max_zip = 11697

df_yelp['zip_code'] = df_yelp['zip_code'].astype(int)

In [ ]:
df_yelp = df_yelp[(df_yelp['zip_code'] >= min_zip) & (df_yelp['zip_code'] <= max_zip)]

### Removing low frequency businesses

In order to ensure a stable model, we're only going to consider businesses in our data with a moderate frequency of 15. This number is arbitrary, but considered a decent representation in data science. 

In [ ]:
freq_treshold = 15
zip_counts = df_yelp["zip_code"].value_counts()
low_count_zip_counts = zip_counts[zip_counts < freq_treshold].index
high_count_zip_counts = zip_counts[zip_counts >= freq_treshold].index

In [ ]:
df_yelp = df_yelp[[zip_code in high_count_zip_counts for zip_code in df_yelp["zip_code"]]]

### Parsing Coordinates

In [ ]:
coordinates = "coordinates"
coord_keys = ["latitude", "longitude"]
lat_and_long = df_yelp[coordinates].map(lambda string: get_keys_from_sting_dict(string, coord_keys))

for key in coord_keys:
    df_yelp[key] = [pair[key] for pair in lat_and_long]

df_yelp.drop(columns=[coordinates], inplace = True)

In [ ]:
df_yelp[coord_keys].describe()

Looking at the latitude and longitude, we see some points that do not appear to be in the New York City area, around 40.7 and -73.9, respectively.

#### Wrong latitude

In [ ]:
df_yelp[(df_yelp["latitude"] > 41) | (df_yelp["latitude"] < 40)]

This data point is from Scarsdale NY, which is not within the city limits. This data point will be dropped.

In [ ]:
df_yelp = df_yelp[(df_yelp["latitude"] <= 41) & (df_yelp["latitude"] >= 40)]

#### Wrong longitude

In [ ]:
df_yelp[(df_yelp["longitude"] > -72) | (df_yelp["longitude"] < -75)]

After looking up these businesses, it is clear they were given a positive longitude when they are actually supposed to be negative.

In [ ]:
df_yelp.loc[(df_yelp["longitude"] > -72) | (df_yelp["longitude"] < -75),"longitude"] = np.negative(df_yelp[(df_yelp["longitude"] > -72) | (df_yelp["longitude"] < -75)]["longitude"])

### Parsing categories

The `categories` column is condensed as a dictionary. By selecting the `alias` and reassigning the column as a string of categories, this process prepares the column for later conversion into dummy variables.

In [ ]:
def convert_string_dict_to_string(string, key):
    return ",".join([dic[key] for dic in literal_eval(string)])

df_yelp["categories"] = df_yelp["categories"].map(lambda s: convert_string_dict_to_string(s,"alias"))

### Parsing transactions

Similarly, the `transactions` column is condensed as a list, which will also be parsed and prepped for later conversion into dummy variables.

In [ ]:
def convert_string_list_to_string(string):
    return ",".join(literal_eval(string))

df_yelp["transactions"] = df_yelp["transactions"].map(convert_string_list_to_string)

In [ ]:
df_yelp.shape

In [ ]:
df_yelp.isnull().sum().sum()

There are no null values - this is a complete dataset

### Cleaning IRS Dataset [Hadi]
These data were collected directly from the IRS website ([source](https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-2016-zip-code-data-soi))

In [ ]:
df_irs = pd.read_csv('./data/irs.csv')

In [ ]:
df_irs[10:20]

In [ ]:
yelp_zips = list(set(df_yelp['zip_code']))

In [ ]:
def clean_str_num(str_num):
    '''Returns integer of input string with commas removed'''
    return int(str_num.replace(',',''))

In [ ]:
affluency_rates = []
found_zips = []
missing_zips = []
returns_col_name = 'Number of returns'

for zip_code in yelp_zips:
    try:
        sub_df               = df_irs[df_irs.iloc[:,0] == str(zip_code)]
        
        affluent_irs_returns = clean_str_num(sub_df[returns_col_name].iloc[-1])
        total_irs_returns    = clean_str_num(sub_df[returns_col_name].iloc[0])
        affluent_rate        = affluent_irs_returns / total_irs_returns
        
        affluency_rates.append(affluent_rate)
        found_zips.append(zip_code)
    except Exception as e:
        missing_zips.append(zip_code)
        pass

In [ ]:
len(missing_zips)

There are 26 zip codes in the yelp data that were not found in the IRS dataset.  
These associated datapoint will be dropped, as they have not target value.

In [ ]:
affluency_df = pd.DataFrame(data = {"zip_code": found_zips, "affluency_rate":affluency_rates})
affluency_df.head()

### Merging Yelp and IRS dataset
Merging the yelp dataset with the IRS dataset will drop those observations with missing zip codes.

In [ ]:
df = pd.merge(df_yelp, affluency_df, on = "zip_code")

In [ ]:
df.shape

### Changing Data Type

In [ ]:
df.dtypes

In [ ]:
convert_to_int = ["review_count","rating"]
df[convert_to_int] = df[convert_to_int].astype(int)

In [ ]:
df.head(2)

In [ ]:
df.isnull().sum().sum()

In [ ]:
df.shape

We are left with 9809 complete data points

## Feature Engineering
- [All]

### Interaction term: price\*rating
This interaction term represents a scoring value that allows for relative comparisons between businesses with varying levels of price and rating.

For example: a restaurant with 4 stars and "\\$" price is not as impressive as a restaurant with 4 stars and "\\$\\$\\$\\$" price. 

In [ ]:
df["price*rating"] = df["price"] * df["rating"]

### The `ListColumnExpander` class

- [Mahdi] Explain a bit about how this works and why you made it

### Creating Dummy Variables

In [ ]:
expansion_columns = ["categories","transactions"]
lce = columnExpander.ListColumnExpander(expansion_columns)

dummy_df = pd.DataFrame(lce.fit_transform(df).toarray(), columns=lce.get_feature_names())

complete_df = pd.concat([df.drop(columns = expansion_columns), dummy_df], axis=1)
complete_df.head(2)

### Setting Affluency Threshold
Affluency = when a zip code has 15% of its population file and IRS return of $$200k or more

In [ ]:
affluency_thresh = .15

In [ ]:
complete_df["is_affluent"] = (complete_df["affluency_rate"] >= affluency_thresh).astype(int)
complete_df["is_affluent"].value_counts(normalize = True)

About 30% of all reported IRS returns in New York City count as being affluent, according to our definition.

This leads our data to be somewhat unbalanced, which we need to keep in mind when evaluating our models.

### Reducing Latitude and Longitude into clusters
We decided to reduce the latitude and longitude variables into a single column using $K$-means clustering as a way of representing more local neighborhoods.

An optimum $k$ will be determined using the model's silhouette score.

In [ ]:
def get_kmean_sil_score(data, k, random_state):
    return silhouette_score(data,KMeans(n_clusters = k, n_jobs = -1, random_state = random_state).fit(data).labels_)

In [ ]:
#Setting up K-means scree plot
max_k = 14

lat_long_df = StandardScaler().fit_transform(complete_df[["latitude","longitude"]]) # not 100% necessary to scale
scores = []
best_k = {"k":0,"score":0}
all_k  = range(2, max_k + 1)
best_km = None
for k in all_k:
    score = get_kmean_sil_score(lat_long_df, k, random_state)
    scores.append(score)
    if score > best_k["score"]:
        best_k["score"] = score
        best_k["k"] = k

In [ ]:
plt.title("Scree plot of lat/long $k$-means")
sns.scatterplot(all_k, scores)
plt.plot(best_k["k"],best_k["score"], "ro")
plt.text(best_k["k"] + .5 ,best_k["score"], "k = {}\nscore = {:.2%}".format(best_k['k'],best_k['score']));

According to the silhouette score, an optimum number of clusters for longitude and latitude occur is 9.

In [ ]:
km = KMeans(n_clusters= best_k["k"], random_state = random_state)
km.fit(lat_long_df)
complete_df["location_cluster"] = km.labels_

### Creating "Incomplete" Dataset
As a proof of concept and practical use-case, we're interested in allowing a user to input a NYC zip code and quickly know the affluency of the area. This would allow emergency responders to quickly assess which areas are in the most need. This application is explained in more detail in the [Data Query](#Data-Query) section.

Having an incomplete dataset (which includes observations without affluency) allows us to pass the pertinent information about a zip code through our model to make a prediction for the affluency of the area.

In [ ]:
temp_incomplete_df = pd.merge(df_yelp, affluency_df, on = "zip_code", how = 'outer')
temp_incomplete_df[convert_to_int] = temp_incomplete_df[convert_to_int].astype(int)
temp_incomplete_df["price*rating"] = temp_incomplete_df["price"] * temp_incomplete_df["rating"]

dummy_incomplete_df = pd.DataFrame(lce.transform(temp_incomplete_df).toarray(), columns=lce.get_feature_names())

incomplete_df = pd.concat([temp_incomplete_df.drop(columns = expansion_columns), dummy_incomplete_df], axis=1)

incomplete_df["location_cluster"] = km.predict(incomplete_df[["latitude","longitude"]])

## Exploratory Data Analysis
- [Mahdi] and [Hadi] killer graphs and visuals

In [ ]:
# SET THRESHOLD FOR NUMBER OF OBSERVATIONS TO SHRINK IMAGE
order_of_cities = complete_df.groupby("city")["affluency_rate"].mean().sort_values().index

In [ ]:
plt.figure(figsize = (6,28))
sns.boxplot(data = complete_df, y = "city", x = "affluency_rate", orient="h", order=order_of_cities);

Here, we see some correlation between `city` and the affluency rate of the city. 

However, using subject knowledge we know _these city designations are inconsistent and ambiguous_. For example, "New York" and "New York City" and "Manhattan" and "new york" and "New york" could be considered synonymous.

__Therefore we will not be includeing `city` in our modeling process.__

# ===========
# MESSY BELOW HERE

### Visualizing location clusters

In [ ]:
order_of_clusters = complete_df.groupby("location_cluster")["affluency_rate"].mean().sort_values().index

colors = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231', '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', '#fabebe']

neighborhoods = ["Bronx", "Upper Bkln", "Astoria", "Staten Island", "Upper Mhtn", "Lower Bkln", "Lower Mhtn", "Jamaica", "Flushing", "New Rochelle\n&\nYonkers"]

In [ ]:
plt.figure(figsize = (14,8))
g = sns.boxplot(
    data = complete_df, 
    x = "location_cluster", 
    y = "affluency_rate", 
    order = order_of_clusters,
    palette = colors,
);

g.set(xticklabels = np.array(neighborhoods)[order_of_clusters]);

### Visualizing 

Visualizing affluency by location

In [ ]:
plt.figure(figsize = (12,8))
sns.scatterplot(
    data = complete_df, 
    y = "latitude", 
    x = "longitude", 
    hue = "location_cluster",
    hue_order = order_of_clusters,
    palette = colors,
    s = 16, 
    legend = False
);

In [ ]:
# https://matplotlib.org/basemap/api/basemap_api.html
# https://matplotlib.org/basemap/users/geography.html
# https://basemaptutorial.readthedocs.io/en/latest/backgrounds.html#arcgisimage
plt.figure(figsize=(lon_diff * scale, lat_diff * scale))
m = Basemap(
    resolution = None, 
    llcrnrlat = complete_df["latitude"].min() - lat_pad,
    llcrnrlon = complete_df["longitude"].min() - lon_pad,
    urcrnrlat = complete_df["latitude"].max() + lat_pad,
    urcrnrlon = complete_df["longitude"].max() + lat_pad,
    epsg = 3395
)


m.arcgisimage(service ="World_Shaded_Relief", xpixels = 2000)
m.scatter(lon, lat, latlon=True, 
    s = 1,
    c = np.array(colors)[complete_df["location_cluster"]]
);


# ADD COLOR BARS

In [ ]:
# Prepping geograph
lat    = complete_df['latitude'].values
lon    = complete_df['longitude'].values
price  = complete_df['price'].values
rating = complete_df['rating'].values

padding = .05
lat_diff = complete_df["latitude"].max() - complete_df["latitude"].min()
lat_pad = (lat_diff) * padding

lon_diff = complete_df["longitude"].max() - complete_df["longitude"].min()
lon_pad = (lon_diff) * padding

scale = 28

In [ ]:
from mpl_toolkits.basemap import Basemap

# https://matplotlib.org/basemap/api/basemap_api.html
# https://matplotlib.org/basemap/users/geography.html
# https://basemaptutorial.readthedocs.io/en/latest/backgrounds.html#arcgisimage
plt.figure(figsize=(lon_diff * scale, lat_diff * scale))
m = Basemap(
    resolution = None, 
    llcrnrlat = complete_df["latitude"].min() - lat_pad,
    llcrnrlon = complete_df["longitude"].min() - lon_pad,
    urcrnrlat = complete_df["latitude"].max() + lat_pad,
    urcrnrlon = complete_df["longitude"].max() + lat_pad,
    epsg = 3395
)


m.arcgisimage(service ="World_Shaded_Relief", xpixels = 2000)
m.scatter(lon, lat, latlon=True, 
    s = 1,
    cmap ='cool', 
    c = complete_df["affluency_rate"]
);

# Add colorbar

# MESSY ABOVE HERE
# ===========

## Saving/Loading Clean data

#### Saving cleaned datasets

In [ ]:
# complete_df.to_csv("./data/clean_data.csv")
# incomplete_df.to_csv("./data/clean_incomplete_data.csv")

#### Loading cleaned datasets

In [ ]:
import time
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import columnExpander
from ast import literal_eval
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, f1_score, make_scorer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
import xgboost as xgb

import warnings
warnings.filterwarnings("ignore")

In [ ]:
complete_df = pd.read_csv("./data/clean_data.csv", index_col = 0)
incomplete_df = pd.read_csv("./data/clean_incomplete_data.csv", index_col = 0)

In [ ]:
incomplete_df.shape

In [ ]:
complete_df.shape

## Modeling: Two Paths
With almost 300 features, we were interested in finding an optimal model to determine how significant Yelp prices are in predicting affluency in an area******* (This is different from affluency of individual business)

We considered feature elimination as well as extraction, and decided on splitting the difference. We tried fitting models without altering the data, [PATH 1](#PATH-1:-Default-dataset), as well as trying to fit models with a reduced dataset, [PATH 2](#PATH-2:-Feature-Reduction).

### Establishing baseline model

In [ ]:
complete_df["is_affluent"].value_counts(normalize = True)

The baseline accuracy for the data is about 71.7% being not affluent.

**** Being mindful of our metric - accuracy not what we're using

### Defining scorer object

In order to have GridSearchCV properly optimize for our metric, we have to create a sklearn scorer object and provide it as the `scorer` argument.

In [ ]:
def spec(y_true,y_pred):
    TN, FP, FN, TP = confusion_matrix(y_true,y_pred).ravel()
    specificity = (TN)/(TN+FP)
    return specificity

specificity_scorer = make_scorer(spec)

### Defining Receiving Operating Characteristics Curve Function

We want to define a the function to draw the ROC curve for each model for a visual evaluation for their performance.

## PATH 1: Default dataset
### Model Preparation

In [ ]:
remove_columns = [
    'id',
    'zip_code',
    'city',
    'state',
    'latitude',
    'longitude',
    'affluency_rate',
    'transactions_',
    'location_cluster'
]
target = 'is_affluent'

In [ ]:
X = complete_df.drop(columns=remove_columns+[target])
y = complete_df[target]

In [ ]:
X.shape

### Custom `gridSearchHelper` function
This function simply includes some "quality of life" functionality to help streamline the modeling process, such as printing the training score, test, score and and confusion matrix.

In [ ]:
def gridSearchHelper(estimator, params, X, y, standardize = False):
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = random_state, stratify = y)

    if standardize:
        sc      = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test  = sc.transform(X_test)

    gs = GridSearchCV(estimator, params, cv = 5, n_jobs=-1, scoring = specificity_scorer, verbose=2)
    gs.fit(X_train, y_train)
    print("Train score:", gs.best_score_)
    print("Test score:", gs.score(X_test, y_test))
    print("Confusion Matrix:\n", confusion_matrix(y_test, gs.predict(X_test)))
    return gs

### Logistic Model

In [ ]:
lr_params = {'C': [1.023292992280754],
          'penalty': ['l2']}

gridSearchHelper(LogisticRegression(), lr_params, X, y);

Our initial model to predict affluency is the Logistic Model. We want to use the simpliest model as a benchmark to see how it performs. Our model incorporates most of the features engineered with the exception of redundant features removed during the model preparation process. Utilizing gridsearch, we discovered the optimal parameters that gave the highest score for specificity. We want to use specificity as the primary metric because it would not be benefit for improverished neighborhoods to be labeled as affluent in the event of disaster. It can cause the incorrect distribution of resource and aid for those particular neighborhoods.

### K-Nearest Neighbors

In [ ]:
knn_params = {'n_neighbors': [11],
              'algorithm': ['ball_tree']}

gridSearchHelper(
    KNeighborsClassifier(),
    knn_params, 
    X, y, 
    standardize = True # Must standardize for KNN
);

Our next model will be K-Nearest Neighbors where it functions by grouping up data depending on their proximity. The specificity score here did not surpass the Logistic Model but it was not far off. However, because it did not exceed expections, we will not utilize these models.

### Decision Tree

In [ ]:
dt_params = {'max_depth': [3],
             'min_samples_split': [2],
             'min_samples_leaf': [1],
             'max_features': [ None]}

gs_dt = gridSearchHelper(
    DecisionTreeClassifier(random_state = random_state), 
    dt_params, 
    X, y
);

The DecisionTree Model has delivered desireable results as our confusion has showed that we have greatly reduced our false positives and false negatives. In addition to the low amount of false positives, the model also presented low amount of variance which shows that the data provided had many meaningful and important features that provided this analysis with a fair amount of accuracy. As we continue to explore the models below, we will evaluate other tree-type models and see if improvements can be made whether by either tuning parameters or selecting more robust models. However, with the DecisionTree achieving a score that we see above, we might not see much improvements, if any, that can better examine the data and provide additional accuracy.

### Bagged Trees

In [ ]:
bc_params = {'n_estimators': [10, 20, 30],
             'max_features': [10, 20, 30, 40, 50]}

model_bg = gridSearchHelper(
    BaggingClassifier(random_state = random_state), 
    bc_params, 
    X, y
);

In [ ]:
model_bg.best_params_

As suspected, we generally should not increase complexity of modeling with ensemble methods if DecisionTree already provided great results. Here we see although, the amount of false positives has been greatly reduced, it was at the expense of accuracy and a huge increase in false negatives. Even if the goal is to optimize for specificity, the inbalance of false values is a great concern for the merits of this model. Since Bagging Trees' result kind of regressed, it would not be beneficial to proceed with the other variations that involve bootstrapping and we predict those models, RandomForest, and Extra Trees, will not yeild any interesting results.

### XGBoost

In [ ]:
boost_params = {'booster': ['gbtree'],
             'max_depth': [3],
             'learning_rate': [.01]}

model_xg = gridSearchHelper(
    xgb.XGBClassifier(random_state = random_state), 
    boost_params, 
    X, y
);

In [ ]:
model_xg.best_params_

One final model we wanted to attempt was the XGBoost (Extreme Gradient Boost) model. While the model performed great, it did not provide any improvement over the DecisionTree model used above. Furthermore, the cores appeared to mirror the DecisiontTree. Although XGBoost was proclaimed to be an extremely robust model that has been proven to be very accurate when used, in this scenario it unforunately did not deliver.

## PATH 2: Feature Reduction
### Model Preparation

use `interact`  
to make predictive zip code affluency function

## Data Query

With the model of choice in hand, we want to create a query for our client to access to quickly determine if a particular area is affluent so that resources can be distributed properly in times of disaster. Below, a user can input either a zip code of choice or city and it will return that area with its affluency status attached.

In [ ]:
incomplete_df[incomplete_df['zip_code'] == 11249]

In [ ]:
model = gridSearchHelper(
    BaggingClassifier(random_state = random_state), 
    bc_params, 
    X, y);

We instantiate the best model above in preparation for creating the query function.

In [ ]:
def query(user_input):
    if type(user_input) == int: # the first check in the function would be to see if the input is a string or an integer
                                # this is to enable the query take both type of values and return their appropriate result
            
        try: # We want to filter out any non-NYC cities or zipcodes and return a message that this query will only work for NYC area
            
            if incomplete_df[incomplete_df['zip_code'] == user_input]['affluency_rate'].isnull().sum() == 0:
            # For integeer type data, the input will be a zipcode; we want to check if that particular zip code
            # already has the affluency rate generated from the IRS data above. If the data exist, we can directly
            # pull the data and display it from the dataframe
                
                return pd.DataFrame({'zip_code': user_input, 
                                     'affluency_rate': incomplete_df[incomplete_df['zip_code'] == user_input]['affluency_rate'].iloc[0]}, 
                                    index = range(1))
                # If the above condition is met, the function will return a dataframe for that particular zipcode        
                
            else: # If the zipcode given did not have affluency rate given, we can use our model above to predict
                  # the affluency rate
                    
                X_incomplete_dummy = incomplete_df[incomplete_df['zip_code'] == user_input]
                # We are instantiating a dummy local datagrame for all data within that particular zipcode
                
                X_zip_column = X_incomplete_dummy[X.columns]
                # Because of the nature of the model (XGBoost), it is required that the features MUST be in the same order
                # for both fitted data and predicting data. We can solve that by setting the features in the same order as our
                # model by retrieving the features from model preparation section above
                
                y_pred = model.predict(X_zip_column)
                # We retrieve our predict values from the incomplete file and instantiate that to a variable
                
                return pd.DataFrame({'zip_code': user_input, 'affluency_rate': y_pred.mean()}, 
                                    index= range(1))
                # We return a single row dataframe that outputs the zipcode with its associated affluency rate by taking the 
                # mean of all predicted value (0s and 1s) within the zipcode
        except:
            print('Please enter a valid NYC zipcode.') #returning a prompt for invalid NYC zipcode input
        
    else: # If the name of the city (neighborhood or borough) is given, we can apply a different method to show the
              # affluency rate for all zipcodes within that region
        
        if incomplete_df[incomplete_df['city'] == user_input].shape[0] != 0:
            # We want to filter out cities that are not in NYC with this conditional statement
            
            city_df = incomplete_df[incomplete_df['city'] == user_input]
            # We want to create a sub dataframe here to include only all data within that city

            if city_df['affluency_rate'].isnull().sum() == 0: 
            # Similar to the zipcode application above, if all values of affluency within that data exists, then we will not 
            # need to use our model and predict affluency for that particular region

                zip_code_afflcuency = []
                # Here we are instantiating an empty list to store each observation for zip and affluency within that particular region

                for i in city_df['zip_code'].unique():
                # Because each zip code is repeated multiple times per business, we only need the unique zipcodes within that data

                    zip_code_afflcuency.append({'zip_code': i, 
                                                'affluency_rate': city_df[city_df['zip_code'] == i]['affluency_rate'].iloc[0]})
                    # For each zipcode that it gets looped through, we only need to take the first value for that zipcode range
                    # and append it to the list. The reason for this is the affluency rate for a single zipcode is always the same.
                    # We only need to show each zipcode's affluency rate for a given region queried.

                return pd.DataFrame(zip_code_afflcuency)
                # This will return the list created above into a dataframe showing all the zipcodes and their associated affluency rate

            else: # As per above, we have to consider certain regions where some zipcodes did not have their affluency rate provided
                  # by the IRS data
                city_affluency = []
                # Here we also instantiate an empty list to store the zipcodes and their affluency rate after extraction

                for i in city_df['zip_code'].unique():
                # As above, we want to only look at unique zip codes since there will be repeats throughout the dataframe

                    if city_df[city_df['zip_code'] == i]['affluency_rate'].isnull().sum() == 0:
                    # for the zipcodes that already has affluency rate, we can isolate those values with the condition set above
                        city_affluency.append({'zip_code': i, 
                                               'affluency_rate': city_df[city_df['zip_code'] == i]['affluency_rate'].iloc[0]})
                        # we will store all those values within the empty list instantiated above 

                    else: # for the remaining values (i.e. data that were not given by the IRS) we will use our model to predict
                          # the affluency rate for all zipcodes within that region
                        X_complete = complete_df.drop(columns=remove_columns+[target])
                        # As per above, we need to set the columns in the same order for fitting and predicting
                        
                        X_incomplete_dummy = city_df[incomplete_df['zip_code'] == i]
                        # Instantiating the dataframe for the particular zipcode as a variable
                        
                        X_zip_column = X_incomplete_dummy[X.columns]
                        # Matching the feature order as the model fitted
                        
                        y_pred = model.predict(X_zip_column)
                        # creating an array of predicted values
                        
                        city_affluency.append({'zip_code': i, 'affluency_rate': y_pred.mean()})
                        # Appending the list above with the predicted values as an average of affluency

                return pd.DataFrame(city_affluency)
                # returning the dataframe for that particular region
        
        else: # For cities entered that are not within NYC, the prompt below will appear to alert the user of the error
            print("Please enter a proper city name.")

The query operates with an input of either a zip or a string of the city's name. This will return a dataframe with the necessary information of that area's affluency.

In [ ]:
query(10010) #Query showing affluency rate for Flatiron here at Manhattan

In [ ]:
query('Bronx') # Query showing affluency rate for every zipcode in Bronx

In [ ]:
query('Las Vegas') # An invalid city name will prompt a message to input a proper name

In [ ]:
query(89147) # An invalid zipcode will prompt a message to please input a proper NYC zipcode

## Model Evaluation

We want to evaluate our model that performed the best in terms of specificity and accuracy. Using the query function created from above, we can examine the output for zip codes that did not have an affluency rate given by the IRS data. 

In [ ]:
query(11249)

<img src="./images/11249.png" alt="drawing" width="720"/>

In the query above, the model determined that particular zip code is close to the .15 threshold. Our model predicted this area to have an affluency rate of 11%. We can verify the model [here](https://newyork.hometownlocator.com/zip-codes/data,zipcode,11249.cfm) and see that the average income per household is \\$90,853 with the median household income at \\$57,389. While we are not provided with the proportion of household incomes per income bracket, the area observed is within middle class range.

In [ ]:
query(11239)

<img src="./images/11239.png" alt="drawing" width="720"/>

The query for the zip code above predicted that partcular zip is not affluency for any of the businesses as a data point. A verification can be show via [this link](https://www.zipdatamaps.com/11239) where the median housefhold income is \\$21,116 and the average household income is \\$39,110. With income ranges that low, our model appears to accurately classified this neigborhood to be low income based and would benefit from additional disaster relief aid in times of crisis.

In [ ]:
query(10459)

<img src="./images/10459.png" alt="drawing" width="720"/>

Our final evaluation of the model shows the zip code above to be not affluent. According this [data](https://www.zipdatamaps.com/10455), the average household income is \\$27,060 and the median income is \\$24,406. Similar to the previous neighborhood, this area was classified by our model to be well below our affluence threshold. Providing aid to this neighborhood as a priority would be crucial when disaster occurs.

In [ ]:
query(10020)

<img src="./images/10020.png" alt="drawing" width="720"/>

The last query we want to present here is a commercial zip code (i.e. all the buildings within this zip code do not have residential capabilities). 10020 was predicted to be a highly affluent area, which is correct (This zip code inhibits the Rockefeller Center Building) and would be categorized as high valued. Likewise, businesses around this area would be considered to be high priced consumer services which coresponds to great amount of affluency. In times of disaster, we can confidently say that this area would probably not need immediate aid. 

## Conclusions and Recommendations
- [All]

## Source Documentation
- [NYC zip codes](https://www.nycbynatives.com/nyc_info/new_york_city_zip_codes.php)
- [Yelp API - Business Endpoints](https://www.yelp.com/fusion)
- [IRS dataset](https://www.irs.gov/statistics/soi-tax-stats-individual-income-tax-statistics-2016-zip-code-data-soi)

# TO DO 
- Clean up imports
- Organize the Table of Contents